## N-queens with Ariel
This file demonstrates how to tackle the n-queens problem using Ariel. Solving for a board with n=8, n=16 and n=32 where n is the the board length and width and also the number of queens on the board. The goal is to achieve an average of zero attacking queens on all tested n values. This will demonstrate the power and usability of the Ariel library.

In [ ]:
# Standard library
import random
from collections.abc import Callable
from dataclasses import dataclass
from pathlib import Path
from typing import Literal, cast

# Pretty little errors and progress bars
from rich.console import Console
from rich.progress import track
from rich.traceback import install

# Third-party libraries
import numpy as np
from pydantic_settings import BaseSettings
from sqlalchemy import create_engine
from sqlmodel import Session, SQLModel, col, select 

# Local libraries
from ariel.ec.a000 import IntegerMutator, IntegersGenerator
from ariel.ec.a001 import Individual
from ariel.ec.a005 import Crossover
from ariel.ec.a004 import EASettings, EAStep, EA, AbstractEA

# Library to show fitness landscape
import matplotlib.pyplot as plt

In [ ]:
type Population = list[Individual]
config = EASettings()

In [ ]:
def parent_selection(population: Population, k=2) -> Population:
    """Tournament selection of size k"""
    random.shuffle(population)
    for idx in range(0, len(population) - 1, k):
        ind_i = population[idx]
        ind_j = population[idx + 1]

        # Compare fitness values
        if ind_i.fitness > ind_j.fitness and config.is_maximisation:
            ind_i.tags = {"ps": True}
            ind_j.tags = {"ps": False}
        else:
            ind_i.tags = {"ps": False}
            ind_j.tags = {"ps": True}
    return population


def crossover(population: Population) -> Population:
    """One-point crossover"""
    parents = [ind for ind in population if ind.tags.get("ps", False)]
    for idx in range(0, len(parents), 2):
        parent_i = parents[idx]
        parent_j = parents[idx + 1]  # Fix: use the next parent, not the same one
        genotype_i, genotype_j = Crossover.one_point(
            cast("list[int]", parent_i.genotype),
            cast("list[int]", parent_j.genotype),
        )

        # First child
        child_i = Individual()
        child_i.genotype = genotype_i
        child_i.tags = {"mut": True}
        child_i.requires_eval = True

        # Second child
        child_j = Individual()
        child_j.genotype = genotype_j
        child_j.tags = {"mut": True}
        child_j.requires_eval = True

        population.extend([child_i, child_j])
    return population


def mutation(population: Population) -> Population:
    for ind in population:
        if ind.tags.get("mut", False):
            genes = cast("list[int]", ind.genotype)
            mutated = IntegerMutator.integer_swap(
                individual=genes,
                span=1,
                mutation_probability=0.5,
            )
            ind.genotype = mutated
    return population

def count_attacking_queens(solution: list[int]) -> int:
    """Counts the number of attacking queens on diagonals."""
    fitness = 0
    main_diags, anti_diags = set(), set()
    
    for row, col in enumerate(solution):
       if row - col in main_diags:
          fitness += 1
       if row + col in anti_diags:
          fitness += 1
      
       main_diags.add(row - col)
       anti_diags.add(row + col)       

    return fitness

def evaluate(population: Population) -> Population:
    for ind in population:
        if ind.requires_eval:
            ind.fitness = -count_attacking_queens(cast("list[int]", ind.genotype)) # Negative because we want to minimize attacking queens
    return population

def survivor_selection(population: Population) -> Population:
    random.shuffle(population)
    current_pop_size = len(population)
    for idx in range(len(population)):
        ind_i = population[idx]
        ind_j = population[idx + 1]

        # Kill worse individual
        if ind_i.fitness > ind_j.fitness and config.is_maximisation:
            ind_j.alive = False
        else:
            ind_i.alive = False

        # Termination condition
        current_pop_size -= 1
        if current_pop_size <= config.target_population_size:
            break
    return population


def create_individual(n) -> Individual:
    ind = Individual()
    ind.genotype = IntegersGenerator.integers(low=0, high=n, size=n, shuffle=True)
    return ind


def learning(population: Population) -> Population:
    return population